# Belegungsquote der Frauenhäuser in Deutschland

## Load source data

In [1]:
import pandas as pd
import numpy as np

In [2]:
fpath = "frauenhaus_suche.json"

In [3]:
# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

In [4]:
# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

In [5]:
# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])

In [6]:
# reformat timeseries
df.timestamp = pd.to_datetime(df.timestamp, dayfirst=True)
df["date"] = df.timestamp.dt.date

In [7]:
# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich für Frauen mit bis zu einem Kind",
    "Aufnahme möglich für Frauen ohne Kinder":"Platz für Frauen ohne Kinder",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich ohne detaillierte Angabe",
    "k.A.":"keine Angabe"
}

df["status"] = df.description.map(desc_short)

In [8]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
df = pd.merge(df, geocoded, on=["shelter_name","shelter_id"])

In [9]:
# load metadata
metadata = pd.read_csv("data/helpers/shelters_metadata.csv")
metadata = metadata.drop(['title'], axis=1)

In [10]:
# get monthyear as column
df["monthyear"] = df.timestamp.dt.to_period("M")

In [11]:
# trim dates
df = df.loc[(df.monthyear >= "2022-01") & (df.monthyear <= "2022-12"),]

In [12]:
# get counts of timestamp to use as denominator
counts = df[["monthyear","timestamp"]].drop_duplicates().groupby(["monthyear"]).count().reset_index()
counts = counts.rename(columns={"timestamp":"timestamp_count"})

In [13]:
# and total counts
n = counts.timestamp_count.sum()

In [14]:
# add data completeness and get shelters with data less than 80% of time
data_completeness = df.groupby(["shelter_id"]).timestamp.count().reset_index()
data_completeness["pct_data_availability"] = data_completeness["timestamp"]/n
incomplete = data_completeness.loc[data_completeness.pct_data_availability <= .8, "shelter_id"].values

## Shelter-level data by month

In [15]:
# get count of values per day
df_monthyear = df.groupby(["monthyear","shelter_name","shelter_id","bundesland","bez","gen","status","latitude","longitude"]).agg(
    status_count = ("status", "count")
).reset_index()
df_monthyear = df_monthyear.loc[~df_monthyear.shelter_id.isin(incomplete),] # filter out incomplete, dont need to share with lokalen
df_monthyear.head()

,monthyear,shelter_name,shelter_id,bundesland,bez,gen,status,latitude,longitude,status_count
0,2022-01,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,keine Angabe,53.552828,9.996640,92
1,2022-01,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Keine Aufnahme möglich,51.325188,12.373901,91
2,2022-01,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Platz für Frauen ohne Kinder,51.325188,12.373901,1
3,2022-01,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,Aufnahme möglich für Frauen mit bis zu einem Kind,50.933467,6.998638,6
4,2022-01,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,Aufnahme möglich für Frauen mit mehreren Kindern,50.933467,6.998638,2


In [16]:
# pivot to wide and fill values
df_monthyear_wide = pd.pivot(
    df_monthyear,
    index=["shelter_name","shelter_id","bundesland","bez","gen","monthyear"],
    columns="status",
    values="status_count"
).reset_index().replace(np.nan, 0)
df_monthyear_wide.head()

status,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,0.0,0.0,0.0,0.0,0.0,92.0
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-02,0.0,0.0,0.0,0.0,0.0,80.0
2,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-03,0.0,0.0,0.0,0.0,0.0,91.0
3,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-04,0.0,0.0,0.0,0.0,0.0,85.0
4,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-05,0.0,0.0,0.0,0.0,0.0,90.0


In [17]:
# add total count of timestamp for percent denominator
df_monthyear_wide = pd.merge(df_monthyear_wide, counts, how="outer")
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,timestamp_count
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,0.0,0.0,0.0,0.0,0.0,92.0,92
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,2022-01,0.0,0.0,0.0,91.0,1.0,0.0,92
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2022-01,6.0,2.0,0.0,84.0,0.0,0.0,92
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,2022-01,0.0,0.0,0.0,0.0,0.0,92.0,92
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2022-01,0.0,0.0,0.0,88.0,4.0,0.0,92


In [18]:
# calculate percentages
df_monthyear_wide.iloc[:,6:-1] = df_monthyear_wide.iloc[:,6:-1].apply(lambda x : x / df_monthyear_wide.timestamp_count, axis=0)
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,timestamp_count
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,0.000000,0.000000,0.0,0.000000,0.000000,1.0,92
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,2022-01,0.000000,0.000000,0.0,0.989130,0.010870,0.0,92
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2022-01,0.065217,0.021739,0.0,0.913043,0.000000,0.0,92
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,2022-01,0.000000,0.000000,0.0,0.000000,0.000000,1.0,92
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2022-01,0.000000,0.000000,0.0,0.956522,0.043478,0.0,92


In [19]:
df_monthyear_wide = pd.merge(df_monthyear_wide, metadata, on='shelter_id', how="left")

In [20]:
# drop timestamp camp and save file
df_monthyear_wide = df_monthyear_wide.drop(["timestamp_count"], axis=1)
df_monthyear_wide.to_csv("./data/cleaned/belegungsquote_nach_year_month_shelter.csv", index=False)

## Overall shelter-level summary

In [21]:
# get counts of status by shelter
df_shelter = df.groupby(["shelter_name","shelter_id","bundesland","bez","gen","status","latitude","longitude"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter.head()

,shelter_name,shelter_id,bundesland,bez,gen,status,latitude,longitude,n
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,Aufnahme möglich ohne detaillierte Angabe,53.552828,9.996640,18
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,keine Angabe,53.552828,9.996640,1052
2,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Aufnahme möglich für Frauen mit bis zu einem Kind,51.325188,12.373901,151
3,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Keine Aufnahme möglich,51.325188,12.373901,228
4,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Platz für Frauen ohne Kinder,51.325188,12.373901,691


In [22]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id","bundesland","bez","gen","latitude","longitude"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)

In [23]:
# calculate total unique timestamp counts
n = sum(counts.timestamp_count)

# get columns as pct
df_shelter_wide.iloc[:,-6:] = df_shelter_wide.iloc[:,-6:]/n

df_shelter_wide.head()

status,shelter_name,shelter_id,bundesland,bez,gen,latitude,longitude,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.996640,0.000000,0.000000,0.016822,0.000000,0.000000,0.983178
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,51.325188,12.373901,0.141121,0.000000,0.000000,0.213084,0.645794,0.000000
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,50.933467,6.998638,0.005607,0.003738,0.000935,0.986916,0.002804,0.000000
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,52.516227,13.377745,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,50.921396,6.995416,0.001869,0.011215,0.000000,0.982243,0.004673,0.000000


In [24]:
df_shelter_wide = pd.merge(df_shelter_wide, metadata, on='shelter_id')

In [25]:
# filter out incomplete data and save to csv
df_shelter_wide.loc[~df_shelter_wide.shelter_id.isin(incomplete),].to_csv("./data/cleaned/belegungsquote_nach_shelter.csv", index=False)

## Filter to only Frauenhäuser with less than 20% keine Angabe and 80% or more of the time data

In [26]:
good_data = df_shelter_wide.loc[(df_shelter_wide["keine Angabe"] < 0.20) & (df_shelter_wide.einrichtungsart=="Frauenhaus"), "shelter_id"].values
good_data = [x for x in good_data if x not in incomplete]

In [27]:
df_monthyear_wide["datenlage_gut"] = df_monthyear_wide.apply(lambda row : row.shelter_id in good_data, axis=1)

In [28]:
datenlage = df_monthyear_wide[["bundesland","shelter_id","datenlage_gut"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_shelters = ("shelter_id", "count"),
    n_datenlage_gut = ("datenlage_gut", sum)
).reset_index()
datenlage = datenlage.rename(columns={
    "n_shelters":"Gesamtzahl der Frauenhäuser",
    "n_datenlage_gut":"Anzahl der Frauenhäuser, die wir analysieren könnten"
})
datenlage

,bundesland,Gesamtzahl der Frauenhäuser,"Anzahl der Frauenhäuser, die wir analysieren könnten"
0,Baden-Württemberg,41,21
1,Bayern,37,19
2,Berlin,10,0
3,Brandenburg,10,4
4,Bremen,2,0
5,Hamburg,5,0
6,Hessen,27,27
7,Mecklenburg-Vorpommern,9,5
8,Niedersachsen,24,10
9,Nordrhein-Westfalen,62,62


## Filter monthly data by good data quality and type: Frauenhaus

In [29]:
# filter no data or only frauenhaus
df_monthyear_wide = df_monthyear_wide.loc[df_monthyear_wide.datenlage_gut==True,]

## Bundesweit monthly overview

In [30]:
# remove shelters with no data
monthly = df_monthyear_wide \
    .groupby(["monthyear"]) \
    .mean() \
    .reset_index()
monthly = monthly.drop(["shelter_id","Gehbehinderung","Hörbehinderung/Taubheit","Sehbehinderung/Blindheit","Suchtmittelabhängigkeit","datenlage_gut"], axis=1)
monthly.head()

,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,2022-01,0.017201,0.114299,0.080345,0.741652,0.026446,0.009133
1,2022-02,0.020855,0.093240,0.063202,0.789541,0.027806,0.003571
2,2022-03,0.018073,0.071289,0.061527,0.811067,0.030290,0.005020
3,2022-04,0.022156,0.060615,0.041983,0.834996,0.028665,0.011586
4,2022-05,0.014834,0.067682,0.055725,0.840496,0.016526,0.004738


In [31]:
monthly.to_csv("./data/cleaned/monthly_overview.csv", index=False)

## Bundesland-level summary

In [32]:
# filter out where no data and mean of each status % by bundesland
bundesland = df_monthyear_wide \
    .groupby(["bundesland"]) \
    .mean() \
    .reset_index()

bundesland = bundesland.drop(["shelter_id","Gehbehinderung","Hörbehinderung/Taubheit","Sehbehinderung/Blindheit","Suchtmittelabhängigkeit","datenlage_gut"], axis=1)
bundesland = pd.merge(bundesland, datenlage, on="bundesland", how="outer")
bundesland.head()

,bundesland,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,Gesamtzahl der Frauenhäuser,"Anzahl der Frauenhäuser, die wir analysieren könnten"
0,Baden-Württemberg,0.017102,0.082782,0.017027,0.835650,0.006338,0.032801,41,21
1,Bayern,0.012684,0.060258,0.073669,0.819829,0.018658,0.014567,37,19
2,Brandenburg,0.102753,0.189024,0.013361,0.658260,0.000260,0.036343,10,4
3,Hessen,0.008792,0.025705,0.018844,0.940036,0.006387,0.000000,27,27
4,Mecklenburg-Vorpommern,0.013087,0.039745,0.127389,0.767201,0.027666,0.023463,9,5


In [33]:
bundesland.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Bundesland and month-level summary

In [34]:
# filter out where no data and mean of each status % by bundesland
bundesland_monthyear = df_monthyear_wide \
    .groupby(["bundesland", "monthyear"]) \
    .mean() \
    .reset_index()

bundesland_monthyear = bundesland_monthyear.drop(["shelter_id","Gehbehinderung","Hörbehinderung/Taubheit","Sehbehinderung/Blindheit","Suchtmittelabhängigkeit","datenlage_gut"], axis=1)
bundesland_monthyear = pd.merge(bundesland_monthyear, datenlage, on="bundesland", how="outer")
bundesland_monthyear.head()

,bundesland,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,Gesamtzahl der Frauenhäuser,"Anzahl der Frauenhäuser, die wir analysieren könnten"
0,Baden-Württemberg,2022-01,0.007065,0.244022,0.016304,0.711957,0.000000,0.020652,41,21
1,Baden-Württemberg,2022-02,0.058750,0.173125,0.000000,0.748750,0.019375,0.000000,41,21
2,Baden-Württemberg,2022-03,0.016745,0.054945,0.016222,0.836211,0.007849,0.042386,41,21
3,Baden-Württemberg,2022-04,0.017367,0.091317,0.006723,0.814006,0.006723,0.063866,41,21
4,Baden-Württemberg,2022-05,0.017989,0.059259,0.041799,0.826455,0.010053,0.044444,41,21


In [35]:
bundesland_monthyear = bundesland_monthyear.loc[bundesland_monthyear.monthyear.notnull(),]

In [36]:
bundesland_monthyear.to_csv("./data/cleaned/monthly_nach_bundesland.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [37]:
# filter by data and get relevant variables
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","status","gen","bez","bundesland"]]

In [38]:
# add metadata
weihnacht = pd.merge(weihnacht, metadata, on='shelter_id')

In [39]:
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)